In [ ]:
# Сразу набор команд для создания фичей из timestamp
test['reviewTime_new'] = pd.to_datetime(test.unixReviewTime, unit='s')
train['review_year'] = pd.DatetimeIndex(train['review_time']).year
train['review_month'] = pd.DatetimeIndex(train['review_time']).month
train['review_day'] = pd.DatetimeIndex(train['review_time']).day
train['review_dayofweek'] = pd.DatetimeIndex(train['review_time']).dayofweek.astype(object)

In [ ]:
# Изменить порядок колонок
cols = ['review_dayofweek']  + [col for col in train if col != 'review_dayofweek']

In [ ]:
# Этот код выбирает числовые признаки, вычисляет их логарифмы, выбирает два категориальных признака,
# применяет к ним one-hot кодирование и объединяет оба множества в одно, а таже суммирует значения признаков между собой.

import itertools

def itr_combinations_new_features(data):
    
    tmp_df = data.copy()
    
    old_columns = [item for item in tmp_df.columns.tolist() if item != 'rating' and item != 'review_dayofweek' and item != 'userid']
    
    tmp_df_sum_2 = [(pd.Series(tmp_df.loc[:,list(i)].sum(axis=1),\
            name='_sum2_'.join(tmp_df.loc[:,list(i)].columns))) for i in list(itertools.combinations(old_columns,2))]
    
    tmp_df_sum_3 = [(pd.Series(tmp_df.loc[:,list(i)].sum(axis=1),\
            name='_sum3_'.join(tmp_df.loc[:,list(i)].columns))) for i in list(itertools.combinations(old_columns,3))]
    
    tmp_df_multiplication = [(pd.Series(tmp_df.loc[:,i[0]]*tmp_df.loc[:,i[1]],\
            name='_mult_'.join(tmp_df.loc[:,list(i)].columns))) for i in list(itertools.combinations(old_columns,2))]
    
   
    df_sum_2 = pd.DataFrame(tmp_df_sum_2).T  
    
    df_sum_3 = pd.DataFrame(tmp_df_sum_3).T  
    
    df_multiplication = pd.DataFrame(tmp_df_multiplication).T  
   
    data = pd.concat([data, df_sum_2, df_sum_3, df_multiplication], axis=1, sort=False)         
 
    
    return data


train_full = itr_combinations_new_features(train_full)

In [ ]:
# Сразу сделаю функцию для проверки и работе с выбросами
def outliers_iqr_short(ys):
    # Определяет номера значений с отклонением больше, чем iqr
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - 1.5*iqr
    upper_bound = quartile_3 + 1.5*iqr
    return np.where((ys > upper_bound) | (ys < lower_bound))[0]

def outliers_iqr_long(ys): # Передаем на вход признак-столбец датафрейма
    # Находим необходимые параметры
    median = ys.median()
    quartile_1, quartile_3 = np.percentile(ys, [25, 75])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    print('Медиана: {},'.format(median),'25-й перцентиль: {},'.format(quartile_1), '75-й перцентиль: {},'.format(quartile_3)
      , "IQR: {}, ".format(iqr),"Границы выбросов: [{}, {}].".format(lower_bound,upper_bound))
    # Отбрасываем значения, лежещие за пределами границ, рисуем график
#     ys.loc[ys.between(lower_bound, upper_bound)].hist(bins = int(upper_bound-lower_bound), range = (lower_bound, upper_bound), label = 'IQR')
#     plt.legend();
    # На выход отдаем отфильтрованные значения
    
    first_rec = ys.mask((ys > upper_bound) | (ys < lower_bound))
    second_rec = np.where((ys > upper_bound) | (ys < lower_bound))[0]
    
    return first_rec


In [1]:
# Функция очистки от выбросов
def delete_outliers_iqr(df, column):
    # Считаем первый и третий квартили
    first_quartile = df[column].describe()['25%']
    third_quartile = df[column].describe()['75%']

    # IQR (Межквартильный размах)
    iqr = third_quartile - first_quartile

    print(first_quartile, third_quartile, iqr)

    # Удаляем то, что выпадает за границы IQR
    #     df_tmp = df.copy()
    df = df[(df[column] > (first_quartile - 3 * iqr)) &
                (df[column] < (third_quartile + 3 * iqr))]

    df[column].hist()
    df[column].describe()

    df = df.loc[df[column].between(first_quartile - 1.5*iqr, third_quartile + 1.5*iqr)]
    df.info()

In [ ]:
# Функция для создания нового признака на базе группировки
def cat_age(x):
    if x == 1 : x = 1 # типа, если автомобилю 1 год, то добавим его в категорию 1
    elif 2 <= x < 4: x = 2 # типа, если автомобилю от 2 до 3 лет включительно, то добавим его в категорию 2 и тд.
    elif 4 <= x < 6: x = 3
    elif 6 <= x < 10: x = 4
    elif 10 <= x < 16: x = 5
    elif 16 <= x < 20: x = 6
    elif 20 <= x: x = 7
    return x 


test['car_age'] = test['car_age'].apply(lambda x: cat_age(x))

test['car_age'] = test['car_age'].astype('str')

In [ ]:
# Замена категориальных/бинарных на числовые
test['custom_clear'] = test['custom_clear'].apply(lambda x: "1" if x == "Растаможен" else "0").astype(int)

In [ ]:
# Новый датафрейм с разбитыми данными из нужно мне колонки - сделаю так, чтобы это было решением не частным, а для всех случаев
body_type_tmp = test['body_type'].str.split(" ", n = 1, expand = True) 

In [ ]:
# Замена по словарю
# словарь для кодировки
dic_body_type = {
    'внедорожник': 'allroad',
    'кабриолет': 'cabriolet',
    'компактвэн': 'compact_van',
    'купе': 'coupe',
    'лифтбек': 'liftback',
    'родстер': 'roadster',
    'седан': 'sedan',
    'универсал': 'station_wagon',
    'хэтчбек': 'hatchback'
}

test['body_type'] = test['body_type'].map(dic_body_type)

In [ ]:
# конъюнкция двух признаков
def make_conj(data, feature1, feature2):
    data['tmp_1'] = np.where(~data[feature2].str.contains('xDrive|Competition|sDrive|Package'),
                                 '', data[feature2])
    
    data['tmp_0'] = data[feature1]
    
    data['tmp'] = data['tmp_0'] + ' ' + data['tmp_1']
                    
    return (data)

# выполним корректировку названия
make_conj(name_tmp, 0, 1)

In [ ]:
# Напишем функцию, которая сделает полный список всех возможных фич в тестовой выборке.
def get_test_features(equipment):
    # Создаем пустой список, в который будут добавляться все фичи
    all_features = []
    for data in equipment:
        # Находим все слова между кавычками
        features=re.findall(r'\"(.+?)\"',data)
        # Добавляем в общий список
        all_features.extend(features)
    # Удаляем дубликаты
    all_features = list(dict.fromkeys(all_features))
    return all_features

test_features = get_test_features(test.equipment)

# Удаляем лишние записи
for bad_feature in ['name','Безопасность','values','Комфорт','Мультимедиа','Обзор','Салон','Защита от угона','Элементы экстерьера']:
    test_features.remove(bad_feature)    
print('Всего уникальных фич:', len(test_features))
print(test_features)

def get_features_test(equipment): 
    features=re.findall(r'\"(.+?)\"',equipment)  
    return features

In [ ]:
# Сделаем отбор фич с коэффициентом корреляции > 0.3
cor_target = abs(train_equipment.corr()["price"])
relevant_features = cor_target[cor_target>0.3] # Выбираем фичи с значением модуля коэффициента корреляции > 0.3
relevant_features

In [ ]:
#plot graph of feature importances for better visualization

plt.figure(figsize = (12,8))
feat_importances = pd.Series(etc.feature_importances_, index=X.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.show()

In [ ]:
# Посмотрим вообще от каких признаков колеблется цена. Может там что-то интересного найду.
for c in new_train.columns[:-1]:
    plt.figure(figsize=(20,5))
    plt.title("{} vs. \nprice".format(c),fontsize=16)
    plt.scatter(x=new_train[c],y=new_train['price'],color='blue',edgecolor='k')
    plt.grid(True)
    plt.xlabel(c,fontsize=14)
    plt.ylabel('Стоимость', fontsize=14)

    plt.show()

In [ ]:
new_train_corr = new_train.corr()
new_train_corr

# new_train_corr.style.background_gradient(cmap='coolwarm')
new_train_corr.style.background_gradient(cmap='coolwarm').set_precision(2)

# Create correlation matrix
corr_matrix = new_train.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

print(to_drop)

# Drop features 
# new_big_df.drop(to_drop, axis=1, inplace=True)

In [ ]:
Q1 =  new_train['price'].quantile(0.25)
Q3 = new_train['price'].quantile(0.75)
IQR = Q3 - Q1

new_train.query("@Q1 - 1.5*@IQR < price < @Q3 + 1.5*@IQR").boxplot(column="price")

In [ ]:
# Проверим на выбросы цены
fig = plt.figure(figsize=(7, 7))
plt.grid(True)
plt.title('Price')
sns.boxplot(x = new_train['price'])

In [ ]:
# Тепловая карта
sns.set(font_scale=1)
plt.subplots(figsize=(10, 10))
sns.heatmap(fuel_type_corr, square=True, 
           annot=True, fmt=".1f", linewidths=0.1, cmap="RdBu")

In [ ]:
# Формирование dummy- столбцов для топлива
new_train_extend = pd.concat([new_train, 
                              pd.get_dummies(new_train['fuel_type'])], 
                             axis=1)